<a href="https://colab.research.google.com/github/im-sohail/Python/blob/main/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Dense,Flatten,Layer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.utils import np_utils
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
x_train , y_train = tfds.as_numpy(tfds.load('horses_or_humans',split = 'train', batch_size = -1, as_supervised = True))

Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteRNMZ4X/horses_or_humans-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteRNMZ4X/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [ ]:
x_test , y_test = tfds.as_numpy(tfds.load('horses_or_humans',split = 'test', batch_size = -1, as_supervised = True))

In [ ]:
print(x_train.shape,x_test.shape)

(1027, 300, 300, 3) (256, 300, 300, 3)


In [ ]:
print(y_train.shape)

(1027,)


In [ ]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
x_train = tf.image.resize(x_train,[224,224])
x_test = tf.image.resize(x_test,[224,224])

In [ ]:
x_train.shape

TensorShape([1027, 224, 224, 3])

In [ ]:
y_train = np_utils.to_categorical(y_train,2)
y_test = np_utils.to_categorical(y_test,2)

In [ ]:
vgg = VGG16(include_top = True, weights = 'imagenet')

553467904/553467096 [==============================] - 5s 0us/step


In [ ]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
for layer in vgg.layers:
  layer.trainable= False

In [ ]:
vgg_input = vgg.layers[0].input
vgg_output = vgg.layers[-2].output
predictions = Dense(2)(vgg_output)


In [ ]:
model=Model(inputs=vgg_input , outputs=predictions)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [ ]:
model.fit(x_train,y_train ,epochs = 5)

Epoch 1/5
33/33 [==============================] - 36s 82ms/step - loss: 7.6256 - accuracy: 0.5341
Epoch 2/5
33/33 [==============================] - 2s 68ms/step - loss: 7.5411 - accuracy: 0.5083
Epoch 3/5
33/33 [==============================] - 2s 68ms/step - loss: 7.7391 - accuracy: 0.4954
Epoch 4/5
33/33 [==============================] - 2s 68ms/step - loss: 7.3385 - accuracy: 0.5215
Epoch 5/5
33/33 [==============================] - 2s 68ms/step - loss: 7.4236 - accuracy: 0.5160


In [ ]:
model.evaluate(x_test,y_test)

8/8 [==============================] - 1s 69ms/step - loss: 7.6685 - accuracy: 0.5000


[7.6685471534729, 0.5]